In [1]:
import yfinance as yf
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.test import SMA

# ----------------------
# Download the Data
# ----------------------
df_raw = yf.download(["SPY", "UPRO"], start="2010-01-01", end="2023-01-01", progress=False)

# ------------------------------
# Extract SPY & UPRO Using .xs()
# ------------------------------
df_spy = df_raw.xs('SPY',  level=1, axis=1)
df_upro = df_raw.xs('UPRO', level=1, axis=1)

# -------------------------------
# Rename Columns Correctly
# -------------------------------
df_spy.columns = ['Open_SPY', 'High_SPY', 'Low_SPY', 'Close_SPY', 'Volume_SPY']

# ---------------------------
# Merge SPY and UPRO Data
# ---------------------------
df_merged = pd.concat([df_spy, df_upro], axis=1)
df_merged.dropna(inplace=True)

df_merged.reset_index(inplace=True)  # Ensure the Date index is a normal column if needed


c:\Users\minhp\AppData\Local\Programs\Python\Python313\Lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

C:\Users\minhp\AppData\Local\Temp\ipykernel_5124\2898966120.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_raw = yf.download(["SPY", "UPRO"], start="2010-01-01", end="2023-01-01", progress=False)


In [2]:
df_merged

,Date,Open_SPY,High_SPY,Low_SPY,Close_SPY,Volume_SPY,Close,High,Low,Open,Volume
0,2010-01-04,85.515625,85.560897,84.142305,84.791237,118944600,2.082251,2.086988,2.033039,2.034092,102592800
1,2010-01-05,85.742035,85.779766,85.153470,85.462847,111579900,2.100145,2.103566,2.059882,2.081198,108986400
2,2010-01-06,85.802391,86.013670,85.591112,85.659021,116074400,2.106198,2.119094,2.089225,2.093830,107920800
3,2010-01-07,86.164574,86.270213,85.402454,85.643917,131091100,2.131856,2.138171,2.075803,2.093566,76939200
4,2010-01-08,86.451279,86.489010,85.764621,85.938170,126402800,2.152646,2.154751,2.101725,2.113435,89388000
...,...,...,...,...,...,...,...,...,...,...,...
3267,2022-12-23,370.189240,370.334250,365.471355,367.037531,59857300,32.201717,32.260303,31.020273,31.420597,12097000
3268,2022-12-27,368.729401,370.421263,367.037538,370.073237,51638200,31.840458,32.279839,31.391314,32.201725,8897400
3269,2022-12-28,364.146881,370.653313,363.914864,368.661721,70911500,30.649242,32.318888,30.590659,31.820925,11445900
3270,2022-12-29,370.701630,371.581402,366.486459,367.018205,66970900,32.211479,32.465342,31.186257,31.322954,14159700


In [3]:
# -------------------------------------
# Define the Strategy
# -------------------------------------
class LeveragedForTheLongRun(Strategy):
    n = 100  # SMA period

    def init(self):
        # Use SPY's Close price for the SMA calculation
        self.sma_underlying = self.I(SMA, self.data.df['Close_SPY'], self.n)

    def next(self):
        spy_close = self.data.df['Close_SPY'].iloc[self.data.index[-1]]
        sma_value = self.sma_underlying[-1]  # Latest SMA value

        # If SPY > SMA, Buy UPRO; Otherwise, close position
        if spy_close > sma_value:
            if not self.position:
                self.buy()
        else:
            if self.position:
                self.position.close()

# ------------------------------------------
# Run the Backtest
# ------------------------------------------
bt = Backtest(
    df_merged,
    LeveragedForTheLongRun,
    cash=100_000,
    commission=.0005,
    trade_on_close=False
)

C:\Users\minhp\AppData\Local\Temp\ipykernel_5124\649758606.py:26: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(


In [4]:
stats = bt.run()
print(stats)

Start                                     0.0
End                                    3271.0
Duration                               3271.0
Exposure Time [%]                     76.6198
Equity Final [$]                 650379.89291
Equity Peak [$]                 1428759.69404
Commissions [$]                   23607.03037
Return [%]                          550.37989
Buy & Hold Return [%]              1781.89498
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Alpha [%]                          -125.32626
Beta                                  0.37921
Max. Drawdown [%]                   -54.49753
Avg. Drawdown [%]                    -4.40799
Max. Drawdown Duration                  636.0
Avg. Drawdown Duration               24.28226
# Trades                                 65.0
Win Rate [%]                      

In [5]:
bt.plot()

GridPlot(id='p1354', ...)